In [6]:
import numpy as np
import os
import xml.dom.minidom
import xml.etree.cElementTree as ET
import copy
from PIL import Image
from datetime import datetime
import time

In [129]:
class Place():
    place_counter = -1
    """
    Eingangsdaten:
    Klassenpfad, Ausgangsbildgrösse in px, Anzahl Bilder Channel
    
    Konstruktor genieriert ein dictonary, welches wie folgt aussieht:
    
    {Pfad_Klassen:{Klassenname_1:{0:Bildname_1, 1:Bildname1}}
                 :{Klassenname_2:{0:Bildname_1, 1:Bildname2}}
    }
    
    2 Varianten wie die einzelen Zeichen platziert werden können:
    
    1. rdm Verteilung
    2. ....
    
    """
    
    def __init__(self, path, xml_path, segemented_image_path, size, depth, iou_threshold):
        self._path = path
        self._xml_path = xml_path
        self._segemented_image_path = segemented_image_path
        self._size = size
        self._depth= depth
        self._iou_threshold = iou_threshold
        self._dictonary = {'path':{'folder_name':{0:'image_name'}}}
        self._segmented_image = np.zeros((self._size, self._size, self._depth))
        self._bounding_boxes = np.array([0,0,0,0])
        self._body_name = ''
        self._date_time = ''
        self._placed_image_path = ''
        self._x_rdm_min = 0
        self._y_rdm_min = 0
        self._x_rdm_max = 0
        self._y_rdm_max = 0
        self._dict_folders = ''
        self._number_dict_folders = 0
        self._number_dict_images = 0
        self._rdm_image = 0
        self._filename = ''
        
        path_1, dirs_1, files_1 = next(os.walk(self._path))
        self._dictonary = {path_1:{}}
        for folder in dirs_1:
            self._dictonary[path_1][folder] = {}
#             print(folder)
            path_2, dirs_2, files_2 = next(os.walk(path_1 + folder))
            for i, files in zip(range(len(files_2)), files_2):
                self._dictonary[path_1][folder][i] = {files}
#                 print(files) 


    def get_head(self):
        print('get_head')
        now = datetime.now()
        self._date_time = now.strftime("%Y%m%d_%H%M%S")
        slice_position_r = self._placed_image_path.rfind('/')
        slice_position_l = self._placed_image_path[:slice_position_r].rfind('/')
        self._filename = self._date_time + self._filename
        folder = self._placed_image_path[slice_position_l+1:slice_position_r]
        
        string_head =   '<annotation>'\
                        '<folder>'+str(folder)+'</folder>'\
                        '<filename>'+str(self._filename)+'</filename>'\
                        '<path>'+str(self._placed_image_path)+'</path>'\
                        '<source>'\
                        '<database>Unknown</database>'\
                        '</source>'\
                        '<size>'\
                            '<width>'+str(self._size)+'</width>'\
                            '<height>'+str(self._size)+'</height>'\
                            '<depth>'+str(self._depth)+'</depth>'\
                        '</size>'\
                        '<segmented>0</segmented>'\
                        '</annotation>'
        return string_head

    
    def get_body(self,image_name,xmin,ymin,xmax,ymax):
        print('get_body')
        tmp_slice_r = self._placed_image_path.rfind('_tailored')
        tmp_slice_l = self._placed_image_path[:tmp_slice_r].rfind('_')
        self._filename = self._filename + '_' + self._placed_image_path[tmp_slice_l+1:tmp_slice_r]
        slice_position = image_name.find('_')
        name = image_name[slice_position-1:slice_position]
        string_object = '<object>'\
                        '<name>'+str(name)+'</name>'\
                        '<pose>Unspecified</pose>'\
                        '<truncated>0</truncated>'\
                        '<difficult>0</difficult>'\
                        '<bndbox>'\
                            '<xmin>'+str(xmin)+'</xmin>'\
                            '<ymin>'+str(ymin)+'</ymin>'\
                            '<xmax>'+str(xmax)+'</xmax>'\
                            '<ymax>'+str(ymax)+'</ymax>'\
                            '</bndbox>'\
                        '</object>'
        return string_object

    
    def iou_check(self):
        print('iou_check')
        if np.count_nonzero(self._segmented_image) == 0:
            self._bounding_boxes[0] = self._x_rdm_max
            self._bounding_boxes[1] = self._y_rdm_max
            self._bounding_boxes[2] = self._x_rdm_min
            self._bounding_boxes[3] = self._y_rdm_min
            tmp_counter = 0
            result = True
        else:
            bounding_box = np.array([0,0,0,0])
            bounding_box[0] = self._x_rdm_max
            bounding_box[1] = self._y_rdm_max
            bounding_box[2] = self._x_rdm_min
            bounding_box[3] = self._y_rdm_min
            length = int(len(self._bounding_boxes)/4)
            
            tmp_iou = np.array([])
            
            for i in range(length):
                tmp_start = 0+(4*i)
                tmp_end = 3+(4*i)
                
                print(tmp_start,tmp_end)
                tmp_bounding_boxes = self._bounding_boxes[tmp_start:tmp_end+1]
                tmp_iou[i] = self.bb_intersection_over_union(bounding_box, tmp_bounding_boxes)
                #########################################################################
                ############
                ##############################
                ######################
            if iou <= self._iou_threshold:
                np.append(self._bounding_boxes,bounding_box)
                print('bounding: ',self._bounding_boxes)
                tmp_counter = 0
                result = True
            else:
                if tmp_counter == 3:
                    result = False
                else:
                    tmp_counter += 1
                    self.place(self)         
        return result
    
    
    def place(self):
        print('place')
        with Image.open(self._placed_image_path) as image:
            img = np.array(image)
            y,x,z = img.shape
#             print(y,x,z)
  
            x_max = self._size - x - 5
            y_max = self._size - y - 5
            self._x_rdm_min = np.random.randint(5,x_max)
            self._y_rdm_min = np.random.randint(5,y_max)
            self._x_rdm_max = self._x_rdm_min + x
            self._y_rdm_max = self._y_rdm_min + y
            result = self.iou_check()
            
            
            if result == True:
                self._segmented_image[self._y_rdm_min:self._y_rdm_max,self._x_rdm_min:self._x_rdm_max,:] = self._segmented_image[self._y_rdm_min:self._y_rdm_max,self._x_rdm_min:self._x_rdm_max,:] + img
                return True   
            else: 
                Place.place_counter += 1 
                return False
        
        
    def create_xml_segmented_image(self, string):
        print('create_xml_segmented_image')
        dom = xml.dom.minidom.parseString(string)
        pretty_xml_as_string_body = dom.toprettyxml()
        pretty_xml_as_string_body_without_version = pretty_xml_as_string_body[23:]
        filepath = self._xml_path + '/' + self._filename + '.xml'
        with open(filepath, 'w+') as file:
            file.write(pretty_xml_as_string_body_without_version)
            
        
        img_uint8 = Image.fromarray(np.uint8(self._segmented_image))
        img_uint8.save(self._segemented_image_path +'/'+self._filename+'.bmp')
        self._segmented_image = np.zeros((self._size, self._size, self._depth))
        self._filename = ''
            
        
    def create_xml_body(self,):
        print('create_xml_body')
        slice_position = self._placed_image_path.find('_')
        object_name = self._placed_image_path[:slice_position]
        xml_name_tmp = self.get_body(self._placed_image_path,self._x_rdm_min-1,self._y_rdm_min-1,self._x_rdm_max+1,self._y_rdm_max+1)
        self._body_name = self._body_name + xml_name_tmp
#         print(self._body_name)
              
    def create_xml(self):
        print('create_xml')
#         name = str(Place.placeIt_counter) + 
#         print(xml_head)
        if Place.place_counter == 3:
            xml_head = self.get_head()
            slice_position = xml_head.find('</annotation>')
            complete_xml = xml_head[:slice_position] + self._body_name + xml_head[slice_position:]
    #         print(complete_xml)

            self.create_xml_segmented_image(complete_xml)
            self._body_name = ''     
        else:
            self.rdm()
    
    def delete_dict(self): 
        print('delete')
        del self._dictonary[self._path][self._dict_folders][self._rdm_image]
        if len(list(self._dictonary[self._path][self._dict_folders]))== 0:
            del self._dictonary[self._path][self._dict_folders]
        elif len(list(self._dictonary[self._path])) == 0:
            return True
        else:
            return False
    
    def rdm(self):
        time.sleep(1)
        print('rdm')
        ### Zufälliger Ordner von Zeichen z.B. A oder B ###
        self._dict_folders = list(self._dictonary[self._path].keys())
        self._number_dict_folders = len(self._dict_folders)
        
        if self._number_dict_folders != 0:
            rdm_folder = np.random.randint(self._number_dict_folders)
            self._dict_folders = self._dict_folders[rdm_folder]
            
            ### Zufälliges Bild von oben bestimmten Zeichen ###
            dict_images = list(self._dictonary[self._path][self._dict_folders].keys())
#             print('dict_images: ',dict_images)
            self._number_dict_images = len(dict_images)
            rdm_element = np.random.randint(self._number_dict_images)
            self._rdm_image = dict_images[rdm_element]
            dict_image_name = str(self._dictonary[self._path][self._dict_folders][self._rdm_image])

            r_tmp = dict_image_name.find('}')
            dict_image_name = dict_image_name[2:r_tmp-1]
    #         print(dict_image)

            ### Bild Pfad des Zeichens ###
            self._placed_image_path = self._path + self._dict_folders + '/' +  dict_image_name
            Place.place_counter +=1

            placed = self.place()
    #         print(self._segmented_image, x_rdm_min,y_rdm_min,x_rdm_max,y_rdm_max, placed)

            if placed == True:
                self.create_xml_body()    
                finish = self.delete_dict()
                if finish == True:
                    print('finish')
                else:
                    self.create_xml()
                    self.rdm()
    
    
    def bb_intersection_over_union(self, boxA, boxB):
        print('bb_intersection_over_union')
        print('xA: ',boxA)
        print('xB: ',boxB)
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])
        
        # compute the area of intersection rectangle
        interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
        # compute the area of both the prediction and ground-truth
        # rectangles
        boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
        boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
        # compute the intersection over union by taking the intersection
        # area and dividing it by the sum of prediction + ground-truth
        # areas - the interesection area
        iou = interArea / float(boxAArea + boxBArea - interArea)
        # return the intersection over union value
        print('iou: ', iou)
        return iou    
    
        
        
#     def create_xml(self):    
#     def segmented_image(self):

In [130]:
obj33 = Place('C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/','C:/Users/User/Desktop/Test/GeneratedData/XML_Files/','C:/Users/User/Desktop/Test/GeneratedData/SegmentedImages',512,3,0)

In [131]:
obj33.rdm()

rdm
place
iou_check
create_xml_body
get_body
delete
create_xml
rdm
place
iou_check
length:  1
0 3
bb_intersection_over_union
xA:  [106 476  28 389]
xB:  [458 331 402 257]
iou:  0.0
bounding:  [458 331 402 257]
create_xml_body
get_body
delete
create_xml
rdm
place
iou_check
length:  1
0 3
bb_intersection_over_union
xA:  [112 435  52 382]
xB:  [458 331 402 257]
iou:  0.0
bounding:  [458 331 402 257]
create_xml_body
get_body
delete
create_xml
rdm
place
iou_check
length:  1
0 3
bb_intersection_over_union
xA:  [192 196  35  44]
xB:  [458 331 402 257]
iou:  0.0
bounding:  [458 331 402 257]
create_xml_body
get_body
delete
create_xml
get_head
create_xml_segmented_image
rdm
place
iou_check
create_xml_body
get_body
delete
create_xml
rdm
place
iou_check
length:  1
0 3
bb_intersection_over_union
xA:  [487 317 394 165]
xB:  [505 287 415 191]
iou:  0.0
bounding:  [505 287 415 191]
create_xml_body
get_body
delete
create_xml
rdm
rdm
rdm
rdm
rdm
rdm


In [ ]:
np.append(self._bounding_boxes,bounding_box)

In [135]:
a = np.array([])
b = np.array([4,5,6,7])
c = np.append(a,b)
print(c[0])

4.0


In [101]:
print(obj33._segmented_image)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [66]:
print(obj33._rdm_image)
print(obj33._filename)
# print(obj33._dictonary[obj33._path][obj33._dict_folders][obj33._rdm_image])
print(obj33._dictonary)
# dict_image_name = str(obj33._dictonary[obj33._path][obj33._dict_folders][obj33._rdm_image])

0
_5_3
{'C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/': {}}


In [39]:
dicti = {'C':{'a':{0:{'hallo'}},'b':{0:{'hallo1'}}}}
print(dicti)
print(len(dicti[]))
# del dicti['C']['a'][0]
# print(dicti)
# print(len(list(dicti['C']['a'])))
# del dicti['C']['a']
# print(dicti)
# del dicti['C']['b']
# print(dicti)
# print(len(dicti['C']))

{'C': {'a': {0: {'hallo'}}, 'b': {0: {'hallo1'}}}}
{'C': {'a': {}, 'b': {0: {'hallo1'}}}}
0
{'C': {'b': {0: {'hallo1'}}}}
{'C': {}}
0


In [27]:
# print(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/']['A_upperCase'][2])
# del obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/']['A_upperCase'][2]
print(obj33._dictonary)

{'C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/': {'A_upperCase': {}, 'B_upperCase': {0: {'B_upperCase_arial_3_tailored.BMP'}, 1: {'B_upperCase_DotsAllForNow_4_tailored.BMP'}, 2: {'B_upperCase_MTCORSVA_5_tailored.BMP'}}}}


In [80]:
a = [0,1,2]
print(len(a))

3


In [5]:
for classes in list(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/']):
    class_1 = list(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/'])
    number_images = len(list(list(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/'][class_1[0]])))
    if number_images != len(list(list(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/'][classes]))):
        print('Ungleichgewicht: Nicht gleichviele Bilder von jeder Klasse vorhanden, siehe Klasse: ' + classes)
        break
#     print('alles i.O')
#     print(class_1[0])
#     print(number_images)
#     print(classes)
# # len(list(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/']))
# list(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/'])

Ungleichgewicht: Nicht gleichviele Bilder von jeder Klasse vorhanden, siehe KlasseB_upperCase


In [5]:
print(list(obj33._dictonary))

['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/']


In [7]:
print(len(list(obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/']['A_upperCase'])))
del obj33._dictonary['C:/Users/User/Desktop/Test/GeneratedData/TailoredImages/']['A_upperCase'][2]

2


KeyError: 2

In [19]:
def get_head():
    string_head =   '<annotation>'\
                    '<folder>'+str('hallo')+'</folder>'\
                    '<filename>'+str('filename')+'</filename>'\
                    '<path>'+str('path')+'</path>'\
                    '<source>'\
                    '<database>Unknown</database>'\
                    '</source>'\
                    '<size>'\
                        '<width>'+str('self._size')+'</width>'\
                        '<height>'+str('self._size')+'</height>'\
                        '<depth>'+str('self._depth')+'</depth>'\
                    '</size>'\
                    '<segmented>0</segmented>'\
                    '</annotation>'
    return string_head

a = get_head()
a.find('</annotation>')
print(a[240:])
print(a[:240])

</annotation>
<annotation><folder>hallo</folder><filename>filename</filename><path>path</path><source><database>Unknown</database></source><size><width>self._size</width><height>self._size</height><depth>self._depth</depth></size><segmented>0</segmented>


In [22]:
dom = xml.dom.minidom.parseString(a)
pretty_xml_as_string_body = dom.toprettyxml()

In [26]:
a = {'C':{0:{'a':0}}}
print(len(a['C'][0]))
del a['C'][0]
print(len(a))
print(len(a['C']))

1
1
0


In [2]:
a = 'C:/Users/User/Desktop/Ablaufdatum/ablaufdatum_selber/all/20180528_200104.jpg'

In [10]:
b = a.rfind('/')
print(a[b+1:])
c = a[:b].rfind('/')
print(c)
print(a[c+1:b])

20180528_200104.jpg
52
all


In [53]:
a = GenerateFolderStructur()

set_folder_structur
set_character_folder
__init__


In [17]:
class Coordinates:
    def __init__(self):
        print('whatever')
    
    def distToPoint(self,a):
        a +=1
        if a <=2:
            self.distToPoint(a)
            return a
            
        """a
        Use pythagoras to find distance
        (a^2 = b^2 + c^2)
        """
        print('disToPoint')

    def isNear(self,a):
        a +=1
        d = self.distToPoint(a)
        print('isNear')
        

In [18]:
obj123 = Coordinates()
a = 0
obj123.isNear(a)

whatever
disToPoint
isNear
